In [6]:
import pandas as pd
import numpy as np

import os

In [12]:
# đọc dữ liệu
diemthi_df = pd.read_csv("Data/diemthi_21-24.csv", encoding="utf-8")
diemthi_df

,SBD,Sở GD,Toán,Văn,Sử,Địa,Lí,Hoá,Sinh,Ngoại ngữ,GDCD,Năm thi
0,2000001,Sở GD&ĐT Thành Phố Hồ Chí Minh,8.0,7.00,NaN,NaN,7.25,NaN,5.25,8.2,NaN,2024
1,2000002,Sở GD&ĐT Thành Phố Hồ Chí Minh,6.0,7.25,6.75,5.75,NaN,NaN,NaN,6.6,7.25,2024
2,2000003,Sở GD&ĐT Thành Phố Hồ Chí Minh,9.2,7.75,NaN,NaN,8.50,NaN,8.00,9.8,NaN,2024
3,2000004,Sở GD&ĐT Thành Phố Hồ Chí Minh,8.2,7.50,7.25,7.25,NaN,NaN,NaN,9.4,8.50,2024
4,2000005,Sở GD&ĐT Thành Phố Hồ Chí Minh,7.8,6.50,4.50,5.00,NaN,NaN,NaN,9.0,8.75,2024
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,2002059,Sở GD&ĐT Thành Phố Hồ Chí Minh,6.0,4.00,NaN,NaN,6.25,NaN,3.00,7.2,NaN,2021
7996,2002060,Sở GD&ĐT Thành Phố Hồ Chí Minh,6.2,6.50,4.00,6.50,NaN,NaN,NaN,7.8,9.00,2021
7997,2002061,Sở GD&ĐT Thành Phố Hồ Chí Minh,6.8,8.00,3.75,6.75,NaN,NaN,NaN,7.2,9.50,2021
7998,2002062,Sở GD&ĐT Thành Phố Hồ Chí Minh,8.0,7.00,NaN,NaN,6.00,NaN,5.75,7.6,NaN,2021
